In [1]:
import tensorflow.keras as keras
import numpy as np

# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

4422102/4422102 [==============================] - 0s 0us/step


In [2]:
# Change the orientation of the dataset inorder to work

In [3]:
import numpy as np
from scipy.signal import correlate2d, convolve2d

class Convolution:
    
    def __init__(self, input_shape, filter_size, num_filters):
        input_height, input_width = input_shape
        self.num_filters = num_filters
        self.input_shape = input_shape
        
        # Size of outputs and filters or kernels or weigths
        
        self.filter_shape = (num_filters, filter_size, filter_size) # (3,3)
        self.output_shape = (num_filters, input_height - filter_size + 1, input_width - filter_size + 1)
        
        self.filters = np.random.randn(*self.filter_shape)
        self.biases = np.random.randn(*self.output_shape)
        
        
    def cross_correlation2D(self, input_mat, filter_mat):
        # Get the dimensions of the image and kernel
        image_height, image_width = input_mat.shape
        # (3, 3) shape
        kernel_height, kernel_width = filter_mat.shape

        # Calculate the output dimensions
        output_height = image_height - kernel_height + 1
        output_width = image_width - kernel_width + 1

        # Create an empty array to store the cross-correlation result
        result = np.zeros((output_height, output_width))

        # Perform cross-correlation
        for i in range(output_height):
            for j in range(output_width):
                patch = input_mat[i:i + kernel_height, j:j + kernel_width]
                result[i, j] = np.sum(patch * filter_mat)
        return result
        
        
    def forward(self, input_data):
        self.input_data = input_data
        # Initialized the input value
        output = np.zeros(self.output_shape)
        for i in range(self.num_filters):
            output[i] = correlate2d(self.input_data, self.filters[i], mode="valid")
        #Applying Relu Activtion function
        output = np.maximum(output, 0)
        return output 
    
    
    def backward(self, dL_dout, lr):
        # Create a random dL_dout array
        dL_dinput = np.zeros_like(self.input_data)
        dL_dfilters = np.zeros_like(self.filters)
        
        for i in range(self.num_filters):
                # Calculating the gradient of the filters or weights
                dL_dfilters[i] = correlate2d(self.input_data, dL_dout[i],mode="valid")
                
                # Calculating the gradient of the inputs
                dL_dinput += convolve2d(dL_dout[i], self.filters[i], mode="full")
                
        self.filters -= lr * dL_dfilters
        self.biases -= lr * dL_dout
    
        
        return dL_dinput


class MaxPool:
    
    def __init__(self, pool_size):
        self.pool_size = pool_size
        
    def forward(self, input_data):
        self.input_data = input_data
        self.num_channels, self.input_height, self.input_width = input_data.shape
        self.output_height = self.input_height // self.pool_size
        self.output_width = self.input_width // self.pool_size
        
        self.output = np.zeros((self.num_channels, self.output_height, self.output_width))
        

        for c in range(self.num_channels):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    start_i = i * self.pool_size
                    start_j = j * self.pool_size

                    end_i = start_i + self.pool_size
                    end_j = start_j + self.pool_size 
                    patch = input_data[c, start_i:end_i, start_j:end_j]

                    self.output[c, i, j] = np.max(patch)
                        
        return self.output
    
    def backward(self, dL_dout, lr):
        dL_dinput = np.zeros_like(self.input_data)
        
        for c in range(self.num_channels):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    start_i = i * self.pool_size
                    start_j = j * self.pool_size

                    end_i = start_i + self.pool_size
                    end_j = start_j + self.pool_size 
                    patch = self.input_data[c, start_i:end_i, start_j:end_j]
                    
                    mask = patch == np.max(patch)
                    
                    dL_dinput[c,start_i:end_i, start_j:end_j] = dL_dout[c, i, j] * mask
                    
        return dL_dinput
                        
class Fully_Connected:
    
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        #self.ouptut_neurons = output_neurons
        self.weights = np.random.randn(output_size, self.input_size)
        #print(self.weights)
        self.biases = np.random.rand(output_size, 1)
        
    
    def softmax(self, z):
        # Shift the input values to avoid numerical instability
        shifted_z = z - np.max(z)
        # Compute the exponentiated values
        exp_values = np.exp(shifted_z)

        # Compute the sum of exponentiated values using log-sum-exp trick
        sum_exp_values = np.sum(exp_values, axis=0)
        log_sum_exp = np.log(sum_exp_values)

        # Compute the softmax probabilities
        probabilities = exp_values / sum_exp_values

        return probabilities


    def softmax_derivative(self, s):
        return np.diagflat(s) - np.dot(s, s.T)
        
    def forward(self, input_data):
        self.input_data = input_data
        flattened_input = input_data.flatten().reshape(1, -1)
        self.z = np.dot(self.weights, flattened_input.T) + self.biases
        self.output = self.softmax(self.z)          
        return self.output
    
    def backward(self, dL_dout, lr):
        # Calculate the gradient of the loss with respect to the pre-activation (z)
        dL_dy = np.dot(self.softmax_derivative(self.output), dL_dout)
        # Calculate the gradient of the loss with respect to the weights (dw)
        dL_dw = np.dot(dL_dy, self.input_data.flatten().reshape(1, -1))

        # Calculate the gradient of the loss with respect to the biases (db)
        dL_db = dL_dy

        # Calculate the gradient of the loss with respect to the input data (dL_dinput)
        dL_dinput = np.dot(self.weights.T, dL_dy)
        dL_dinput = dL_dinput.reshape(self.input_data.shape)

        # Update the weights and biases based on the learning rate and gradients
        self.weights -= lr * dL_dw
        self.biases -= lr * dL_db

        # Return the gradient of the loss with respect to the input data
        return dL_dinput

    

In [4]:
X_train = train_images[:2000] / 255.0
y_train = train_labels[:2000]

X_test = train_images[2000:3000] / 255.0
y_test = train_labels[2000:3000]

In [5]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_test[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [6]:
conv = Convolution(X_train[0].shape, 6, 1)
pool = MaxPool(2)
full = Fully_Connected(121,10)

def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

def cross_entropy_loss(predictions, targets):
    
    num_samples = 10

    # Avoid numerical instability by adding a small epsilon value
    epsilon = 1e-7
    predictions = np.clip(predictions, epsilon, 1 - epsilon)

    # Calculate the categorical cross-entropy loss
    loss = -np.sum(targets * np.log(predictions)) / num_samples

    return loss

def cross_entropy_loss_gradient(actual_labels, predicted_probs):
    num_samples = actual_labels.shape[0]

    # Calculate the gradient of the cross-entropy loss function
    gradient = -actual_labels / (predicted_probs + 1e-7) / num_samples

    return gradient


def predict(input_sample, conv, pool, full):
    # Forward pass through convolutional and pooling layers
    conv_out = conv.forward(input_sample)
    pool_out = pool.forward(conv_out)

    # Flatten the output feature maps
    flattened_output = pool_out.flatten()

    # Forward pass through fully connected layer
    predictions = full.forward(flattened_output)

    return predictions

def train_network(X, y, conv, pool, full, lr=0.01, epochs=130):
    for epoch in range(epochs):
        total_loss = 0.0
        correct_predictions = 0

        for i in range(len(X)):
            # Forward pass
            conv_out = conv.forward(X[i])
            pool_out = pool.forward(conv_out)
            full_out = full.forward(pool_out)

            # Calculate loss and accuracy
            loss = cross_entropy_loss(full_out.flatten(), y[i])
            total_loss += loss
            
            
            one_hot_pred = np.zeros_like(full_out)
            one_hot_pred[np.argmax(full_out)] = 1
            one_hot_pred = one_hot_pred.flatten()
            
            num_pred = np.argmax(one_hot_pred)
            num_y = np.argmax(y[i])
            
            if num_pred == num_y:
                correct_predictions += 1
            # Backward pass
            gradient = cross_entropy_loss_gradient(y[i], full_out.flatten()).reshape((-1, 1))
            full_back = full.backward(gradient, lr)
            pool_back = pool.backward(full_back, lr)
            conv_back = conv.backward(pool_back, lr)

        # Print epoch statistics
        average_loss = total_loss / len(X)
        accuracy = correct_predictions / len(X_train) * 100.0
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {average_loss:.4f} - Accuracy: {accuracy:.2f}%")
        

In [7]:
train_network(X_train, y_train, conv, pool, full)

Epoch 1/130 - Loss: 0.3038 - Accuracy: 10.75%
Epoch 2/130 - Loss: 0.2330 - Accuracy: 10.40%
Epoch 3/130 - Loss: 0.2385 - Accuracy: 9.15%
Epoch 4/130 - Loss: 0.2486 - Accuracy: 7.90%
Epoch 5/130 - Loss: 0.2620 - Accuracy: 10.35%
Epoch 6/130 - Loss: 0.2774 - Accuracy: 12.60%
Epoch 7/130 - Loss: 0.2839 - Accuracy: 15.90%
Epoch 8/130 - Loss: 0.2770 - Accuracy: 17.15%
Epoch 9/130 - Loss: 0.2691 - Accuracy: 19.10%
Epoch 10/130 - Loss: 0.2632 - Accuracy: 20.30%
Epoch 11/130 - Loss: 0.2583 - Accuracy: 22.55%
Epoch 12/130 - Loss: 0.2553 - Accuracy: 24.15%
Epoch 13/130 - Loss: 0.2525 - Accuracy: 25.10%
Epoch 14/130 - Loss: 0.2500 - Accuracy: 25.35%
Epoch 15/130 - Loss: 0.2483 - Accuracy: 25.50%
Epoch 16/130 - Loss: 0.2472 - Accuracy: 26.40%
Epoch 17/130 - Loss: 0.2462 - Accuracy: 26.45%
Epoch 18/130 - Loss: 0.2454 - Accuracy: 26.50%
Epoch 19/130 - Loss: 0.2448 - Accuracy: 26.60%
Epoch 20/130 - Loss: 0.2443 - Accuracy: 26.55%
Epoch 21/130 - Loss: 0.2437 - Accuracy: 26.80%
Epoch 22/130 - Loss: 0.2

In [8]:
predictions = []
for data in X_test:
    pred = predict(data, conv, pool, full)
    one_hot_pred = np.zeros_like(pred)
    one_hot_pred[np.argmax(pred)] = 1
    predictions.append(one_hot_pred.flatten())

predictions = np.array(predictions)

predictions

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [9]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions, y_test)

0.105

In [10]:
for i in range(len(X_test)):

    print("Predictions:",np.argmax(predictions[i]))
    print("Actual:     ", np.argmax(y_test[i]))
    print()

Predictions: 1
Actual:      4

Predictions: 1
Actual:      7

Predictions: 1
Actual:      1

Predictions: 1
Actual:      7

Predictions: 1
Actual:      3

Predictions: 1
Actual:      8

Predictions: 1
Actual:      9

Predictions: 1
Actual:      4

Predictions: 1
Actual:      2

Predictions: 1
Actual:      3

Predictions: 1
Actual:      8

Predictions: 1
Actual:      3

Predictions: 1
Actual:      6

Predictions: 1
Actual:      8

Predictions: 1
Actual:      9

Predictions: 1
Actual:      6

Predictions: 1
Actual:      2

Predictions: 1
Actual:      4

Predictions: 1
Actual:      2

Predictions: 1
Actual:      8

Predictions: 1
Actual:      1

Predictions: 1
Actual:      6

Predictions: 1
Actual:      7

Predictions: 1
Actual:      1

Predictions: 1
Actual:      8

Predictions: 1
Actual:      2

Predictions: 1
Actual:      3

Predictions: 1
Actual:      0

Predictions: 1
Actual:      1

Predictions: 1
Actual:      7

Predictions: 1
Actual:      5

Predictions: 1
Actual:      1

Predicti

In [11]:
conv_out = conv.forward(X_train[0])
pool_out = pool.forward(conv_out)
full_out = full.forward(pool_out)
gradient = cross_entropy_loss_gradient(y_train[0], full_out.flatten()).reshape((-1, 1))
full_back = full.backward(gradient, 0.01)
pool_back = pool.backward(full_back, 0.01)
conv_back = conv.backward(pool_back, 0.01)

In [12]:
13 * 13 * 10

1690